### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
pip install psycopg2-binary sentence-transformers oci pandas pyarrow ipython weasyprint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 149.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 163.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 147.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 287.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 155.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 108.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 155.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 205.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 205.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# --- 1. Imports ---
import os
import oci
import pandas as pd
import psycopg2
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from IPython.display import display
import ipywidgets as widgets

# --- 2. Configuration ---
# OCI Object Storage
BUCKET_NAME = "aus-legal-corpus"
OBJECT_PREFIX = ""
DOWNLOAD_DIR = "./data"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# PostgreSQL Config
DB_CONFIG = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "",
    "host": "10.150.2.103",
    "port": "5432"
}

# OCI Config
oci_config = {
    "user": "ocid1.user.oc1..aaaq",
    "key_file": "./data/oci_api_key.pem",
    "fingerprint": "de:d6",
    "tenancy": "ocid1.tenancy.oc1..aca",
    "region": "ap-melbourne-1"
}

# --- 3. Connect to OCI and Download Parquet Files ---
object_storage = oci.object_storage.ObjectStorageClient(oci_config)
namespace = object_storage.get_namespace().data

print("🔍 Listing objects in bucket...")
objects = object_storage.list_objects(namespace, BUCKET_NAME, prefix=OBJECT_PREFIX).data.objects
parquet_files = [obj.name for obj in objects if obj.name.endswith(".parquet")]

if not parquet_files:
    raise Exception("❌ No .parquet files found. Check bucket, prefix or region.")

for obj_name in parquet_files:
    local_file = os.path.join(DOWNLOAD_DIR, os.path.basename(obj_name))
    if not os.path.exists(local_file):
        print(f"⬇️ Downloading {obj_name} ...")
        with open(local_file, 'wb') as f:
            response = object_storage.get_object(namespace, BUCKET_NAME, obj_name)
            for chunk in response.data.raw.stream(1024 * 1024, decode_content=False):
                f.write(chunk)
print("✅ All Parquet files downloaded.")

# --- 4. Load Embedding Model ---
model = SentenceTransformer("BAAI/bge-small-en-v1.5")
print("✅ Embedding model loaded.")

# --- 5. Connect to PostgreSQL ---
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()

# --- 6. Create Table with Metadata ---
cursor.execute("""
CREATE EXTENSION IF NOT EXISTS vector;

CREATE TABLE IF NOT EXISTS legal_docs_v2 (
    id SERIAL PRIMARY KEY,
    content TEXT,
    jurisdiction TEXT,
    source TEXT,
    citation TEXT,
    embedding VECTOR(384)
);
""")
conn.commit()
print("✅ Table created with metadata.")

# --- 7. Helper: Insert Batch ---
def insert_batch(batch):
    args_str = ",".join(cursor.mogrify("(%s, %s, %s, %s, %s)", x).decode("utf-8") for x in batch)
    cursor.execute("INSERT INTO legal_docs_v2 (content, jurisdiction, source, citation, embedding) VALUES " + args_str)
    conn.commit()

# --- 8. Process Each Parquet File ---
BATCH_SIZE = 10000
local_files = sorted([f for f in os.listdir(DOWNLOAD_DIR) if f.endswith(".parquet")])

for file in local_files:
    print(f"🔄 Processing {file} ...")
    df = pd.read_parquet(os.path.join(DOWNLOAD_DIR, file))

    if "text" not in df.columns:
        print(f"⚠️ Skipping {file}, no 'text' column.")
        continue

    batch = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Embedding & Inserting"):
        try:
            text = row.get("text", "").strip()
            if text == "":
                continue

            vector = model.encode(text).tolist()
            jurisdiction = str(row.get("jurisdiction", "")).strip()
            source = str(row.get("source", "")).strip()
            citation = str(row.get("citation", "")).strip()

            batch.append((text, jurisdiction, source, citation, vector))
            if len(batch) >= BATCH_SIZE:
                insert_batch(batch)
                batch = []
        except Exception as e:
            print(f"❌ Skipping row due to error: {e}")

    if batch:
        insert_batch(batch)

print("✅ All data inserted into PostgreSQL.")

# --- 9. Create Vector Index ---
cursor.execute("""
CREATE INDEX IF NOT EXISTS legal_docs_cosine_idx
ON legal_docs USING ivfflat (embedding vector_cosine_ops)
WITH (lists = 100);
""")
cursor.execute("ANALYZE legal_docs;")
conn.commit()
print("✅ Vector index created.")

# --- 10. Sentence Search Interface ---
def search_query(user_query, top_k=5):
    user_vector = model.encode(user_query).tolist()
    query = """
    SELECT content, jurisdiction, source, citation,
           1 - (embedding <#> %s::vector) AS similarity
    FROM legal_docs
    ORDER BY embedding <#> %s::vector
    LIMIT %s;
    """
    cursor.execute(query, (user_vector, user_vector, top_k))
    rows = cursor.fetchall()
    results = pd.DataFrame(rows, columns=["Content", "Jurisdiction", "Source", "Citation", "Similarity"])
    return results

# --- 11. UI to Accept Query ---
input_box = widgets.Textarea(
    placeholder='Ask a legal question...',
    description='Query:',
    layout=widgets.Layout(width='80%', height='100px')
)

button = widgets.Button(description="Search")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        print("🔍 Searching...")
        result_df = search_query(input_box.value)
        display(result_df)

button.on_click(on_button_click)

display(input_box, button, output)


🔍 Listing objects in bucket...
✅ All Parquet files downloaded.
✅ Embedding model loaded.
✅ Table created with metadata.
🔄 Processing 0000.parquet ...


Embedding & Inserting: 100%|██████████| 11024/11024 [06:05<00:00, 30.17it/s]


🔄 Processing 0001.parquet ...


Embedding & Inserting: 100%|██████████| 11528/11528 [06:19<00:00, 30.39it/s]


🔄 Processing 0002.parquet ...


...

Embedding & Inserting: 100%|██████████| 11848/11848 [06:29<00:00, 30.45it/s]


✅ All data inserted into PostgreSQL.
✅ Vector index created.


Textarea(value='', description='Query:', layout=Layout(height='100px', width='80%'), placeholder='Ask a legal …

Button(description='Search', style=ButtonStyle())

Output()